In [ ]:
print(1)
from qiskit.quantum_info import Statevector, Pauli
from qiskit.circuit.library import HamiltonianGate, HGate
from qiskit.quantum_info import SparsePauliOp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathos.multiprocessing import ProcessingPool
from functools import partial
import multiprocessing

In [ ]:
print(2)
HO=1/np.sqrt(2)*np.array([[1.0, 1.0],[1.0,-1.0]])
UO=1/np.sqrt(2)*np.array([[1.0, 0-1.0j],[1,0+1.0j]])

In [ ]:
print(3)
def init(N):
    stavec = Statevector.from_int(0,2**(N+1))
    for i in range(N+1):
        stavec= stavec.evolve(HO,[i])
    return stavec


def measure_reset(sv):
    sv = sv.evolve(UO,[0])
    meas,sv = sv.measure([0])
    sv=sv.reset([0])
    sv=sv.evolve(HO,[0])
    return sv

In [ ]:
print(4)
def evol_ZZ(sv,U1):
    N = sv.num_qubits-1
    for i in range(N):
        sv=sv.evolve(U1,[0,i+1,(i+1)% N + 1])
        sv = measure_reset(sv)
    sv = measure_reset(sv)
    return sv

def evol_X(sv,U2):
    N = sv.num_qubits-1
    for i in range(1,N+1):
        sv=sv.evolve(U2,[i,0])
        sv = measure_reset(sv)
    return sv 

In [ ]:
print(5)
def Get_ExpectValue_ZZ(sv,N):
    EV_list=[]
    for i in range(int(N/2)):
        EV=[]
        for j in range(N):
            EV.append(np.real(sv.expectation_value(Pauli('ZZ'),[j+1,(j+1+i)%N+1])) ** 2)
        EV_list.append(np.average(EV))
    return EV_list

In [ ]:
print(6)
def Run_ZZX(Num_qubits,X_time,ZZ_time,Sample,Depth):
    N = Num_qubits
    t1 = ZZ_time
    t2 = X_time
    U1 = HamiltonianGate(Pauli('ZZZ'),t1)
    U2 = HamiltonianGate(Pauli('ZX'),t2)
    EV_Sams=[]
    for i in range(Sample):
        sv=init(N)
        EV_Sam=[]
        for j in range(Depth):
            sv=evol_ZZ(sv,U1)
            sv=evol_X(sv,U2)
            EV_Sam.append(Get_ExpectValue_ZZ(sv,N))
        EV_Sams.append(EV_Sam)
    EV_Ave = np.sum(EV_Sams,axis=0) / Sample
#    print(EV_Ave)
    name = "EV_Data/EV_Crit/"+"ZZX_EV"+"+"+str(N)+ "+" + str(t1) + "+" + str(t2) + ".npy" 
    np.save(name,EV_Ave)

In [10]:
print(7)
a=[4,6,8,10,12]
b=[0.095,0.097,0.099,0.101,0.103,0.105]

Num_qubits=[]
X_time=[]

for i in range(5):
    for j in range(6):
        Num_qubits.append(a[i])
        X_time.append(b[j])
print(Num_qubits)
print(np.shape(X_time))

7
[4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 8, 8, 8, 8, 8, 8, 10, 10, 10, 10, 10, 10, 12, 12, 12, 12, 12, 12]
(30,)


In [ ]:
print(8)
if __name__=='__main__':
    Partial_Run0=partial(Run_ZZX,Sample=1000,Depth=50)
    Partial_Run=partial(Partial_Run0,ZZ_time=0.1)
    pool = ProcessingPool(multiprocessing.cpu_count())
    pool.map(Partial_Run, Num_qubits, X_time)
    pool.join()
    pool.close()